Association Rules
-------------------

Association rules analysis is a technique to uncover how items are associated to each other. 

There are three common ways to measure association:

* Measure 1: Support - This says how popular an itemset is, as measured by the proportion of transactions in which an      
    itemset appears.

* Measure 2: Confidence - This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is 
    measured by the proportion of transactions with item X, in which item Y also appears.

* Measure 3: Lift - This says how likely item Y is purchased when item X is purchased, while controlling for how popular item     Y is.

## Apriori algorithm

The Apriori algorithm principle says that if an itemset is frequent, then all of its subsets are frequent.this means that if {0,1} is frequent, then {0} and {1} have to be frequent.

The rule turned around says that if an itemset is infrequent, then its supersets are also infrequent.

We first need to find the frequent itemsets, and then we can find association rules.

In [14]:
## Copied from http://adataanalyst.com/machine-learning/apriori-algorithm-python-3-0/

In [18]:
dataset = [line.split() for line in open('Trafficdata.csv').readlines()]
dataset

[['Year,Case',
  'Vehicle',
  'ID,Vehicle',
  'Body',
  'Type,Registration',
  'Class,Action',
  'Prior',
  'to',
  'Accident,Type',
  '/',
  'Axles',
  'of',
  'Truck',
  'or',
  'Bus,Direction',
  'of',
  'Travel,Fuel',
  'Type,Vehicle',
  'Year,State',
  'of',
  'Registration,Number',
  'of',
  'Occupants,Engine',
  'Cylinders,Vehicle',
  'Make,Contributing',
  'Factor',
  '1,Contributing',
  'Factor',
  '1',
  'Description,Contributing',
  'Factor',
  '2,Contributing',
  'Factor',
  '2',
  'Description,Event',
  'Type,Partial',
  'VIN'],
 ['2013,11256149,2',
  'DOOR',
  'SEDAN,Not',
  'Entered,Backing,Not',
  'Entered,South,Not',
  'Entered,2013,NY,1,,HOND,HUMAN,Backing',
  'Unsafely,HUMAN,Driver',
  'Inattention/Distraction*,Not',
  'Applicable,'],
 ['2013,11330798,4',
  'DOOR',
  'SEDAN,Not',
  'Entered,Overtaking/Passing,Not',
  'Entered,North,Not',
  'Entered,2013,VA,1,,DODG,HUMAN,Unsafe',
  'Lane',
  'Changing,HUMAN,Passing',
  'or',
  'Lane',
  'Usage',
  'Improper,"Overturne

Create a list of candidate itemsets of length k

Scan the dataset to see if each itemset is frequent

Keep frequent itemsets to create itemsets of length k+1

In [3]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [22]:
def apriori_alg(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [5]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList 

The main function is apriori(); it calls aprioriGen() to create candidate itemsets: Ck.

The function aprioriGen() will take a list of frequent itemsets, Lk, and the size of the itemsets, k, to produce Ck. For example, it will take the itemsets {0}, {1}, {2} and so on and produce {0,1} {0,2}, and {1,2}.

The sets are combined using the set union, which is the | symbol in Python

In [6]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict

This function takes three arguments: a dataset, Ck, a list of candidate sets, and minSupport, which is the minimum support you’re interested in. This is the function you’ll use to generate L1 from C1. Additionally, this function returns a dictionary with support values.

In [7]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

Mining association rules from frequent item sets
---------------------------------------------------

To find association rules, we first start with a frequent itemset. We know this set of items is unique, but we want to see if there is anything else we can get out of these items. One item or one set of items can imply another item.

generateRules(), is the main command, which calls the other two.

The generateRules() function takes three inputs: a list of frequent itemsets, a dictionary of support data for those itemsets, and a minimum confidence threshold. It’s going to generate a list of rules with confidence values that we can sort through later.

In [8]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

calcConf() calculates the confidence of the rule and then find out the which rules meet the minimum confidence

In [9]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

rulesFromConseq() generates more association rules from our initial dataset. This takes a frequent itemset and H, which is a list of items that could be on the right-hand side of a rule

In [23]:
L,suppData= apriori_alg(dataset, minSupport=0.3)
suppData

{frozenset({'Entered,Northwest,Not'}): 0.03,
 frozenset({'Entered,2000,NJ,1,,FRHT,HUMAN,Unsafe'}): 0.01,
 frozenset({'2013,11622893,SUBURBAN,Not'}): 0.01,
 frozenset({'Entered,2006,CT,1,,TOYT,HUMAN,Failure'}): 0.01,
 frozenset({'Applicable,"Guide'}): 0.01,
 frozenset({'Applicable,2G1WB5EK7B1'}): 0.01,
 frozenset({"Entered,2010,PA,1,,NISS,ENVMT,Animal's"}): 0.01,
 frozenset({'Applicable,2MG3JM8A7DW'}): 0.01,
 frozenset({'TRUCK,COMMERCIAL,Stopped'}): 0.01,
 frozenset({'Entered,2007,CT,1,,VOLV,ENVMT,View'}): 0.01,
 frozenset({'2013,11161588,4'}): 0.01,
 frozenset({'Closely,Not'}): 0.04,
 frozenset({'Occupants,Engine'}): 0.01,
 frozenset({'Entered,2007,OH,1,,FORD,HUMAN,Unsafe'}): 0.01,
 frozenset({'DOOR', 'SEDAN,Not'}): 0.44,
 frozenset({'Entered,South,Gas,2003,NY,1,10,FORD,HUMAN,Unsafe'}): 0.01,
 frozenset({'Applicable,1ETBX18H8YN'}): 0.01,
 frozenset({'Entered,2013,NJ,1,,FORD,HUMAN,Driver'}): 0.01,
 frozenset({'2013,10950672,SUBURBAN,Not'}): 0.01,
 frozenset({'Entered,2011,CT,1,,HOND,HUM

In [24]:
rules= generateRules(L,suppData, minConf=0.7)

(frozenset(['Entered,Going']), '-->', frozenset(['Ahead,Not']), 'conf:', 1.0)
(frozenset(['Ahead,Not']), '-->', frozenset(['Entered,Going']), 'conf:', 0.888888888888889)
(frozenset(['DOOR']), '-->', frozenset(['Applicable,Not']), 'conf:', 0.7021276595744682)
(frozenset(['SEDAN,Not']), '-->', frozenset(['Applicable,Not']), 'conf:', 0.7045454545454546)
(frozenset(['Applicable,']), '-->', frozenset(['Applicable,Not']), 'conf:', 0.7777777777777777)
(frozenset(['DOOR']), '-->', frozenset(['SEDAN,Not']), 'conf:', 0.9361702127659575)
(frozenset(['SEDAN,Not']), '-->', frozenset(['DOOR']), 'conf:', 1.0)
(frozenset(['Straight']), '-->', frozenset(['Ahead,Not']), 'conf:', 1.0)
(frozenset(['Ahead,Not']), '-->', frozenset(['Straight']), 'conf:', 1.0)
(frozenset(['Applicable,HUMAN,Not']), '-->', frozenset(['Applicable,Not']), 'conf:', 1.0)
(frozenset(['Entered,Going']), '-->', frozenset(['Straight']), 'conf:', 1.0)
(frozenset(['Straight']), '-->', frozenset(['Entered,Going']), 'conf:', 0.88888888888

In [15]:
# Copied from https://github.com/asaini/Apriori/blob/master/apriori.py

In [28]:
%%writefile apriori_prg.py
# Copied from https://github.com/asaini/Apriori/blob/master/apriori.py
import sys
from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser


def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet


def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    toRetRules = []
    for key, value in largeSet.items()[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item)/getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)),
                                           confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key=lambda (item, support): support):
        print "item: %s , %.3f" % (str(item), support)
    print "\n------------------------ RULES:"
    for rule, confidence in sorted(rules, key=lambda (rule, confidence): confidence):
        pre, post = rule
        print "Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)


def dataFromFile(fname):
        """Function which reads from the file and yields a generator"""
        file_iter = open(fname, 'rU')
        for line in file_iter:
                line = line.strip().rstrip(',')                         # Remove trailing comma
                record = frozenset(line.split(','))
                yield record


if __name__ == "__main__":

    optparser = OptionParser()
    optparser.add_option('-f', '--inputFile',
                         dest='input',
                         help='filename containing csv',
                         default=None)
    optparser.add_option('-s', '--minSupport',
                         dest='minS',
                         help='minimum support value',
                         default=0.15,
                         type='float')
    optparser.add_option('-c', '--minConfidence',
                         dest='minC',
                         help='minimum confidence value',
                         default=0.6,
                         type='float')

    (options, args) = optparser.parse_args()

    inFile = None
    if options.input is None:
            inFile = sys.stdin
    elif options.input is not None:
            inFile = dataFromFile(options.input)
    else:
            print 'No dataset filename specified, system with exit\n'
            sys.exit('System will exit')

    minSupport = options.minS
    minConfidence = options.minC

    items, rules = runApriori(inFile, minSupport, minConfidence)

printResults(items, rules)

Writing apriori_prg.py


In [29]:
%%cmd
python apriori_prg.py -f Trafficdata.csv

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

C:\Users\Vishnu\Documents\GitHub\big-data-python-class\Homeworks\Homework7>python apriori_prg.py -f Trafficdata.csv
item: ('SUBURBAN',) , 0.150
item: ('NJ',) , 0.150
item: ('NY', '4 DOOR SEDAN') , 0.150
item: ('East', 'Going Straight Ahead') , 0.150
item: ('', 'SUBURBAN') , 0.150
item: ('NJ', 'HUMAN') , 0.150
item: ('SUBURBAN', 'HUMAN') , 0.150
item: ('Not Applicable', 'Following Too Closely') , 0.150
item: ('Going Straight Ahead', 'Following Too Closely') , 0.150
item: ('North', 'Going Straight Ahead') , 0.150
item: ('', 'NJ') , 0.150
item: ('SUBURBAN', '2013') , 0.150
item: ('Not Entered', 'NJ') , 0.150
item: ('1', 'West') , 0.150
item: ('East', '2013') , 0.150
item: ('SUBURBAN', 'Not Applicable') , 0.150
item: ('NJ', '2013') , 0.150
item: ('Not Entered', 'SUBURBAN') , 0.150
item: ('', 'SUBURBAN', 'HUMAN') , 0.150
item: ('Not Entered', 'SUBURBAN', 'Not Applicable') , 0.150
item: ('Not Entered

In [ ]:
# Copied from http://facweb.cs.depaul.edu/mobasher/classes/CSC478/Notes/IPython%20Notebook%20-%20Associations.html

In [33]:
import apriori as ap

In [34]:
L, support = ap.apriori(dataset,0.5)

In [35]:
print L

[[frozenset(['Entered,Going']), frozenset(['Ahead,Not']), frozenset(['Applicable,Not']), frozenset(['Straight'])], [frozenset(['Ahead,Not', 'Entered,Going']), frozenset(['Ahead,Not', 'Straight']), frozenset(['Straight', 'Entered,Going'])], [frozenset(['Ahead,Not', 'Straight', 'Entered,Going'])], []]


In [36]:
print support

{frozenset(['Entered,Northwest,Not']): 0.03, frozenset(['Entered,2000,NJ,1,,FRHT,HUMAN,Unsafe']): 0.01, frozenset(['2013,11622893,SUBURBAN,Not']): 0.01, frozenset(['Entered,2006,CT,1,,TOYT,HUMAN,Failure']): 0.01, frozenset(['Applicable,"Guide']): 0.01, frozenset(['Applicable,2G1WB5EK7B1']): 0.01, frozenset(["Entered,2010,PA,1,,NISS,ENVMT,Animal's"]): 0.01, frozenset(['Applicable,2MG3JM8A7DW']): 0.01, frozenset(['TRUCK,COMMERCIAL,Stopped']): 0.01, frozenset(['Entered,2007,CT,1,,VOLV,ENVMT,View']): 0.01, frozenset(['2013,11161588,4']): 0.01, frozenset(['Closely,Not']): 0.04, frozenset(['Occupants,Engine']): 0.01, frozenset(['Entered,2007,OH,1,,FORD,HUMAN,Unsafe']): 0.01, frozenset(['Entered,South,Gas,2003,NY,1,10,FORD,HUMAN,Unsafe']): 0.01, frozenset(['Applicable,1ETBX18H8YN']): 0.01, frozenset(['Entered,2013,NJ,1,,FORD,HUMAN,Driver']): 0.01, frozenset(['2013,10950672,SUBURBAN,Not']): 0.01, frozenset(['Entered,2011,CT,1,,HOND,HUMAN,Following']): 0.01, frozenset(['Entered,2004,NJ,1,,CHEV,

In [42]:
ruleList = ap.generateRules(L, support)

frozenset(['Entered,Going']) --> frozenset(['Ahead,Not']) conf: 1.0
frozenset(['Ahead,Not']) --> frozenset(['Entered,Going']) conf: 0.888888888889
frozenset(['Straight']) --> frozenset(['Ahead,Not']) conf: 1.0
frozenset(['Ahead,Not']) --> frozenset(['Straight']) conf: 1.0
frozenset(['Entered,Going']) --> frozenset(['Straight']) conf: 1.0
frozenset(['Straight']) --> frozenset(['Entered,Going']) conf: 0.888888888889
freqSet: frozenset(['Ahead,Not', 'Straight', 'Entered,Going'])
frozenset(['Straight', 'Entered,Going']) --> frozenset(['Ahead,Not']) conf: 1.0
frozenset(['Ahead,Not', 'Entered,Going']) --> frozenset(['Straight']) conf: 1.0
frozenset(['Ahead,Not', 'Straight']) --> frozenset(['Entered,Going']) conf: 0.888888888889
m: 1 Hmp1 now: [frozenset(['Ahead,Not']), frozenset(['Straight']), frozenset(['Entered,Going'])]
Hmp1: [frozenset(['Ahead,Not', 'Straight']), frozenset(['Ahead,Not', 'Entered,Going']), frozenset(['Straight', 'Entered,Going'])]
frozenset(['Entered,Going']) --> frozense

In [43]:
ruleList 

[(frozenset({'Entered,Going'}), frozenset({'Ahead,Not'}), 1.0),
 (frozenset({'Ahead,Not'}), frozenset({'Entered,Going'}), 0.888888888888889),
 (frozenset({'Straight'}), frozenset({'Ahead,Not'}), 1.0),
 (frozenset({'Ahead,Not'}), frozenset({'Straight'}), 1.0),
 (frozenset({'Entered,Going'}), frozenset({'Straight'}), 1.0),
 (frozenset({'Straight'}), frozenset({'Entered,Going'}), 0.888888888888889),
 (frozenset({'Entered,Going', 'Straight'}), frozenset({'Ahead,Not'}), 1.0),
 (frozenset({'Ahead,Not', 'Entered,Going'}), frozenset({'Straight'}), 1.0),
 (frozenset({'Ahead,Not', 'Straight'}),
  frozenset({'Entered,Going'}),
  0.888888888888889),
 (frozenset({'Entered,Going'}), frozenset({'Ahead,Not', 'Straight'}), 1.0),
 (frozenset({'Straight'}),
  frozenset({'Ahead,Not', 'Entered,Going'}),
  0.888888888888889),
 (frozenset({'Ahead,Not'}),
  frozenset({'Entered,Going', 'Straight'}),
  0.888888888888889),
 (frozenset({'Entered,Going'}), frozenset({'Ahead,Not', 'Straight'}), 1.0),
 (frozenset({'

## References

* https://github.com/asaini/Apriori/blob/master/apriori.py

* http://adataanalyst.com/machine-learning/apriori-algorithm-python-3-0/

* http://facweb.cs.depaul.edu/mobasher/classes/CSC478/Notes/IPython%20Notebook%20-%20Associations.html